<a href="https://colab.research.google.com/github/ras0k/list-to-mp3/blob/main/list_to_mp3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# @title List to mp3
!pip install -U yt-dlp
!apt install ffmpeg -y

import subprocess
import os
import ipywidgets as widgets
from IPython.display import display
import requests
from bs4 import BeautifulSoup
import re

# Update yt-dlp to the latest version
try:
    update_result = subprocess.run("yt-dlp -U", shell=True, capture_output=True, text=True)
    print("Updating yt-dlp...")
    print(update_result.stdout)
except Exception as e:
    print("Error updating yt-dlp:", e)

# Function to get playlist name
def get_playlist_title_bs(url):
    headers = {"User-Agent": "Mozilla/5.0"}
    response = requests.get(url, headers=headers)
    if response.status_code != 200:
        print("Failed to retrieve the playlist page.")
        return "merged_audio"
    soup = BeautifulSoup(response.text, "html.parser")
    title_tag = soup.find("title")
    if not title_tag:
        return "merged_audio"
    title = title_tag.text
    # Remove trailing " - YouTube" (or similar) from the title
    title = re.sub(r"\s*-\s*YouTube\s*$", "", title)
    # Sanitize the title by removing or replacing problematic characters
    title = re.sub(r'[\\/*?:"<>|]', "", title)
    # Optionally, replace double slashes or any remaining problematic substrings
    title = title.replace("//", "-")
    return title.strip()

def get_playlist_name(url):
    # Try to use BeautifulSoup to get a clean title
    title = get_playlist_title_bs(url)
    print("Playlist title (sanitized):", title)
    return title

# Function to download audio and merge into a single MP3 using auto-naming
def list_to_mp3(urls):
    print("Splitting input URLs...")
    # Split URLs by new lines and remove any extra whitespace
    url_list = [url.strip() for url in urls.splitlines() if url.strip()]

    # Auto naming: determine output name based on playlist or default to 'merged_audio'
    if len(url_list) == 1 and "playlist?list=" in url_list[0]:
        output_name = get_playlist_name(url_list[0])
    else:
        output_name = "merged_audio"
    print("Output file will be:", output_name + ".mp3")

    # Download audio tracks for each URL with detailed error output
    for idx, url in enumerate(url_list, start=1):
        print(f"Downloading audio for URL {idx}/{len(url_list)}: {url}")
        result = subprocess.run(
            f"yt-dlp -x --audio-format mp3 -o '%(playlist_index)s_%(title)s.%(ext)s' {url}",
            shell=True,
            capture_output=True,
            text=True
        )
        if result.returncode != 0:
            print(f"Error downloading {url}:\nSTDOUT:\n{result.stdout}\nSTDERR:\n{result.stderr}")
            raise Exception("yt-dlp command failed")
        else:
            print(f"Successfully downloaded audio for URL {idx}/{len(url_list)}")

    # Create a file list for merging with ffmpeg using absolute paths
    print("Creating file list for merging...")
    file_list_path = "file_list.txt"
    with open(file_list_path, "w") as f:
        for file in sorted(os.listdir()):
            if file.endswith(".mp3"):
                filepath = os.path.abspath(file)
                f.write(f"file '{filepath}'\n")

    # Build the ffmpeg merge command
    ffmpeg_cmd = f"ffmpeg -f concat -safe 0 -i {file_list_path} -c copy '{output_name}.mp3'"
    print("The ffmpeg command to be executed is:")
    print(ffmpeg_cmd)

    # Merge audio files into a single MP3 file
    print("Merging downloaded audio files into a single MP3...")
    merge_result = subprocess.run(
        ffmpeg_cmd,
        shell=True,
        capture_output=True,
        text=True
    )
    if merge_result.returncode != 0:
        print(f"Error merging files:\nSTDOUT:\n{merge_result.stdout}\nSTDERR:\n{merge_result.stderr}")
        raise Exception("ffmpeg command failed")
    print("Merge completed successfully.")

    # Clean up individual MP3 files and the temporary file list
    print("Cleaning up temporary files...")
    for file in os.listdir():
        if file.endswith(".mp3") and file != f"{output_name}.mp3":
            os.remove(file)
    os.remove(file_list_path)

    return f"✅ Merged audio saved as: {output_name}.mp3"

# Create ipywidgets for input

# TextArea for YouTube URLs (one per line)
text_area = widgets.Textarea(
    value='',
    placeholder='Enter YouTube URLs separated by new lines',
    description='URLs:',
    layout={'width': '100%', 'height': '200px'}
)

# Button to trigger the process
process_button = widgets.Button(
    description='Download & Merge',
    button_style='success'
)

# Output widget to display status messages
output_display = widgets.Output()

# Define the function to be called when the button is clicked
def on_button_click(b):
    with output_display:
        output_display.clear_output()  # clear previous output
        print("Starting processing. Please wait...\n")
        try:
            result = list_to_mp3(text_area.value)
            print("\n" + result)
        except Exception as e:
            print("\nAn error occurred:", e)

process_button.on_click(on_button_click)

# Display the widgets
display(text_area, process_button, output_display)